<a href="https://colab.research.google.com/github/midhunjmes/presidio_final/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
pip install presidio_analyzer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.7 MB/s eta 0:00:00


In [3]:
from presidio_analyzer import AnalyzerEngine

analyzer = AnalyzerEngine()
for recognizer in analyzer.get_recognizers():
    print(recognizer.supported_entities)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


['US_PASSPORT']
['IN_AADHAAR']
['MEDICAL_LICENSE']
['DATE_TIME', 'LOCATION', 'PERSON', 'NRP']
['US_DRIVER_LICENSE']
['CRYPTO']
['PHONE_NUMBER']
['AU_TFN']
['AU_MEDICARE']
['IN_PASSPORT']
['IBAN_CODE']
['US_BANK_NUMBER']
['EMAIL_ADDRESS']
['AU_ACN']
['US_SSN']
['DATE_TIME']
['CREDIT_CARD']
['UK_NHS']
['AU_ABN']
['US_ITIN']
['IN_VOTER']
['URL']
['UK_NINO']
['IP_ADDRESS']
['SG_NRIC_FIN']
['IN_PAN']
['IN_VEHICLE_REGISTRATION']


In [24]:


import pandas as pd
import spacy
import re
import openpyxl
import json
mapping={}
nlp = spacy.load("en_core_web_sm")


#-------------------------------------------------------------------------------------------------------------------------------
#function to find out the noun values dominating columns considering it will be a sensitive data if there is so many nouns
#------------------------------------------------------------------------------------------------------------------------------
def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive


#------------------------------------------------------------------------------------------
#finding out the descriptive data columns which may have sensitive data in the form of text
#------------------------------------------------------------------------------------------
def descriptive_columns(file_path):
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()


    des=[col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]
    return des

#-----------------------------------------------------------------
#anonymizing descriptive values
#-----------------------------------------------------------------
def detect_noun_desc(text):
    doc = nlp(text)
    modified_text=[]
    for token in doc:
        if token.pos_ in ['PROPN']:
            modified_text.append("<sensitive>")
        else:
            modified_text.append(token.text)
    text=" ".join(modified_text)
    return text


#------------------------------------------------------------------
#function for detecting non numerical sensitive
#------------------------------------------------------------------
def detect_sensitive_numerical(file_path,sensitive):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive_columns = []

    for col in df.columns:
        if col not in sensitive:
          text_samples = df[col].astype(str).head(5)  # Convert first 5 values to string

          for value in text_samples:
              if pd.notna(value) and value.strip():
                  results = analyzer.analyze(text=value, language="en")

                  for result in results:
                      # print(result,result.entity_type)
                      if result.entity_type in ["PHONE_NUMBER", "CREDIT_CARD", "IBAN", "US_SSN","EMAIL"]:
                          sensitive_columns.append(col)
                          break  # If any value in the column is sensitive, mark the whole column

    return list(set(sensitive_columns))
#-------------------------------------------------------------------
#function to anonymize a excel file
#-------------------------------------------------------------------
def excel_an(input_file, output_file):
    df = pd.read_excel(input_file)  # Read as string for safety

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))
    num_sensitive=detect_sensitive_numerical(input_file,sensitive_old)
    sensitive=list(set(sensitive)+set(num_sensitive))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column
    mapping = {}

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_excel(output_file, index=False, sheet_name="Anonymized Data")
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------
#function for de-anonymizing excel data
#------------------------------------------------------

def excel_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    df = pd.read_excel(input_file)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_excel(output_file, index=False, sheet_name="De-anonymized Data")

    print(f"✅ De-anonymized file saved as {output_file}")

#----------------------------------------------------------
#function for anonymizing csv data
#----------------------------------------------------------
def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))
    num_sensitive=detect_sensitive_numerical(input_file,sensitive_old)
    sensitive=sensitive+num_sensitive
    column_counters = {col: 1 for col in df.columns if col in sensitive}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_csv(output_file, index=False)
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------------
#function for de anonymizing csv data
#------------------------------------------------------------
def csv_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = pd.read_csv(input_file, engine="python", dtype=str)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_csv(output_file, index=False)

    print(f"✅ De-anonymized file saved as {output_file}")


#--------------------------------------------------------------
#function to determine whcih type of data is need to perform
#--------------------------------------------------------------
def anonymization(input_file):
    if input_file.endswith(".csv"):
        csv_an(input_file,"intermediate.csv")
        csv_dean("intermediate.csv","deanonymized.csv","mappings.json")
    elif input_file.endswith(".xlsx"):
        excel_an(input_file,"intermediate.xlsx")
        excel_dean("intermediate.xlsx","deanonymized.xlsx","mappings.json")

file="numerical.csv"
anonymization(file)

✅ Anonymized file saved as intermediate.csv
🔄 Loading data...
✅ De-anonymized file saved as deanonymized.csv


<ipython-input-24-36990e171947>:241: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)


In [17]:
def detect_sensitive_numerical(file_path,sensitive):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive_columns = []

    for col in df.columns:
        if col not in sensitive:
          text_samples = df[col].astype(str).head(5)  # Convert first 5 values to string

          for value in text_samples:
              if pd.notna(value) and value.strip():
                  results = analyzer.analyze(text=value, language="en")

                  for result in results:
                      # print(result,result.entity_type)
                      if result.entity_type in ["PHONE_NUMBER", "CREDIT_CARD", "IBAN", "US_SSN"]:
                          sensitive_columns.append(col)
                          break  # If any value in the column is sensitive, mark the whole column

    return list(set(sensitive_columns))

In [16]:
sensitive = ['Name', 'Email', 'Address', 'Comments']
detected_columns = detect_sensitive_numerical("numerical.csv", sensitive)
print("Sensitive numerical columns:", detected_columns)

type: UK_NHS, start: 4, end: 14, score: 1.0 UK_NHS
type: DATE_TIME, start: 4, end: 14, score: 0.85 DATE_TIME
type: PHONE_NUMBER, start: 0, end: 14, score: 0.4 PHONE_NUMBER
type: PHONE_NUMBER, start: 0, end: 15, score: 0.4 PHONE_NUMBER
type: DATE_TIME, start: 0, end: 12, score: 0.85 DATE_TIME
type: PHONE_NUMBER, start: 0, end: 12, score: 0.4 PHONE_NUMBER
type: PHONE_NUMBER, start: 0, end: 15, score: 0.4 PHONE_NUMBER
type: DATE_TIME, start: 4, end: 11, score: 0.85 DATE_TIME
type: DATE_TIME, start: 12, end: 17, score: 0.85 DATE_TIME
type: PHONE_NUMBER, start: 0, end: 17, score: 0.4 PHONE_NUMBER
type: CREDIT_CARD, start: 0, end: 19, score: 1.0 CREDIT_CARD
type: CREDIT_CARD, start: 0, end: 19, score: 1.0 CREDIT_CARD
type: CREDIT_CARD, start: 0, end: 18, score: 1.0 CREDIT_CARD
type: CREDIT_CARD, start: 0, end: 19, score: 1.0 CREDIT_CARD
type: CREDIT_CARD, start: 0, end: 19, score: 1.0 CREDIT_CARD
Sensitive numerical columns: ['Phone Number', 'Credit Card']


In [7]:
# import pandas as pd
# import spacy
# import re
# import json

# class DataAnonymizer:
#     def __init__(self, model="en_core_web_sm"):
#         self.nlp = spacy.load(model)
#         self.mapping = {}

#     def detect_noun(self, file_path):
#         df = self._load_data(file_path)
#         sensitive = []

#         for col in df.columns:
#             if df[col].dtype in ['int64', 'float64']:
#                 continue

#             text_samples = df[col].astype(str).head(5)
#             noun_count = sum(
#                 sum(1 for token in self.nlp(value) if token.pos_ in ['NOUN', 'PROPN'])
#                 for value in text_samples
#             )
#             total_count = sum(len(self.nlp(value)) for value in text_samples)

#             if total_count > 0 and (noun_count / total_count) > 0.2:
#                 sensitive.append(col)

#         return sensitive

#     def descriptive_columns(self, file_path):
#         keywords = [
#             "description", "remarks", "notes", "comments", "observations", "details", "summary",
#             "explanation", "reviews", "feedback", "testimonials", "opinions", "assessment",
#             "suggestions", "experience", "incident_report", "case_notes", "audit_notes", "findings",
#             "status_update", "history", "progress_report", "additional_info", "clarifications", "justification",
#             "annotations", "excerpts", "statement", "explanation_text"
#         ]
#         df = self._load_data(file_path, nrows=1)
#         return [col for col in df.columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

#     def detect_noun_desc(self, text):
#         return " ".join("<sensitive>" if token.pos_ == 'PROPN' else token.text for token in self.nlp(text))

#     def anonymize(self, input_file, output_file):
#         df = self._load_data(input_file)
#         sensitive = self.detect_noun(input_file)
#         desc = self.descriptive_columns(input_file)

#         column_mappings = {col: {} for col in sensitive}
#         column_counters = {col: 1 for col in sensitive}

#         for col in sensitive:
#             df[col] = df[col].astype(str).apply(lambda val: self._anonymize_value(val, col, column_mappings, column_counters))

#         for col in desc:
#             df[col] = df[col].astype(str).apply(lambda val: self.detect_noun_desc(val) if pd.notna(val) else val)

#         self._save_data(df, output_file)
#         with open("mappings.json", "w") as f:
#             json.dump(self.mapping, f)
#         print(f"✅ Anonymized file saved as {output_file}")

#     def de_anonymize(self, input_file, output_file, mapping_file):
#         with open(mapping_file, "r") as f:
#             mapping = json.load(f)
#         df = self._load_data(input_file)
#         df = df.applymap(lambda x: mapping.get(x, x))
#         self._save_data(df, output_file)
#         print(f"✅ De-anonymized file saved as {output_file}")

#     def _load_data(self, file_path, **kwargs):
#         if file_path.endswith(".csv"):
#             return pd.read_csv(file_path, engine="python", dtype=str, **kwargs)
#         elif file_path.endswith(('.xls', '.xlsx')):
#             return pd.read_excel(file_path, engine="openpyxl", dtype=str, **kwargs)
#         else:
#             raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

#     def _save_data(self, df, file_path):
#         if file_path.endswith(".csv"):
#             df.to_csv(file_path, index=False)
#         elif file_path.endswith(('.xls', '.xlsx')):
#             df.to_excel(file_path, index=False, sheet_name="Processed Data")

#     def _anonymize_value(self, value, col, column_mappings, column_counters):
#         if pd.notna(value):
#             if value in column_mappings[col]:
#                 anonymized_value = column_mappings[col][value]
#             else:
#                 anonymized_value = f"{col}{column_counters[col]}"
#                 column_mappings[col][value] = anonymized_value
#                 column_counters[col] += 1
#             self.mapping[anonymized_value] = value
#             return anonymized_value
#         return value

# # Example Usage
# anonymizer = DataAnonymizer()
# anonymizer.anonymize("sample_data.xlsx", "anonymized_data.xlsx")
# anonymizer.de_anonymize("anonymized_data.xlsx", "deanonymized_data.xlsx", "mappings.json")


FileNotFoundError: [Errno 2] No such file or directory: 'sample_data.xlsx'